In [8]:
import os
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from ipynb.fs.full.help_functions import number_stances
from sklearn.model_selection import train_test_split
from ipynb.fs.full.modelbuilder import KerasLstmModelBuilder
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

In [9]:
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
      #tf.config.experimental.set_memory_growth(gpu, True)

In [10]:
f = open("preprocessed_text.json", mode = "r", encoding ="utf-8")
d = json.load(f)
f.close()

real_labels = d["stances"]
real_text = d["premises"]

In [11]:
real_labels = np.asarray(real_labels)
real_text = np.asarray(real_text)

d:\miniconda\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [14]:
real_labels[0]

0

In [12]:
real_text[0]

['opponent',
 'forfeited',
 'every',
 'round',
 'none',
 'argument',
 'answered',
 '’',
 'like',
 'idea',
 'winning',
 'default',
 'aretule',
 '’',
 'good',
 'student',
 'get',
 'involved',
 'address',
 'big',
 'issue',
 'like',
 'teen',
 'pregnancy',
 'need',
 'able',
 'answer',
 'argument',
 'like',
 'mine',
 'simply',
 'prepare',
 'abstinenceonly',
 'type',
 'response',
 'also',
 'aware',
 'u',
 'condom',
 'may',
 'sold',
 'minor',
 'state',
 'retailer',
 'say',
 'illegal',
 'sell',
 'frankly',
 'wrong']

In [5]:
corpus_train, corpus_test, Y_train, Y_test = train_test_split(real_text, real_labels, test_size =0.15, random_state = 42, stratify = real_labels)

In [6]:
max_features = 15000
max_len = 150

t = Tokenizer(max_features)
t.fit_on_texts(corpus_train)

word_index = t.word_index

In [7]:
sequence_train = t.texts_to_sequences(corpus_train)
sequence_test = t.texts_to_sequences(corpus_test)

print("Training size is {}".format(len(sequence_train)))
print("Testing size is {}".format(len(sequence_test)))

KeyboardInterrupt: 

In [ ]:
X_train = pad_sequences(sequence_train, padding = "post", maxlen = max_len)
X_test = pad_sequences(sequence_test, padding = "post", maxlen = max_len)

print("Training Sequence Shape: {} \n".format(X_train.shape))
print("Testing Sequence Shape: {} \n".format(X_test.shape))

In [ ]:
lstm_model_builder = KerasLstmModelBuilder(word_index)
model= lstm_model_builder.build_lstm_with_embedding_model(max_features, max_len)

model.summary()

In [ ]:
version = "1.0.5.2"
model_path = "Logs/{}".format(version)
os.mkdir("Logs/{}".format(version))

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=64, validation_data = (X_test, Y_test), epochs=20, callbacks = [model_checkpoint_callback])

In [ ]:
score, acc = model.evaluate(X_test, Y_test, batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("Logs/{}/accuracy".format(version))

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("Logs/{}/loss".format(version))

In [ ]:
#tf.keras.utils.plot_model(
   # model, to_file='model.png', show_shapes=False, show_layer_names=True,
    #rankdir='TB', expand_nested=False, dpi=96
#)

In [ ]:
model_json = model.to_json()

with open("Logs/{}/model.json".format(version), "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("Logs/{}/model_weights.h5".format(version))